In [51]:
import pandas as pd
from recordlinkage.datasets import load_febrl1
data=load_febrl1()
data.head(10)

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
rec-6-dup-0,NaN,trevorrow,16,dumas street,2/98-latchford barracks,mount immaney,2281,wa,19530313,4811642
rec-190-dup-0,darcie,turtur,10,blacket street,eureka,beverly hills,2263,nsw,NaN,2025650
rec-294-org,william,bishop,21,neworra place,apmnt 65,worongary,6225,qld,19490130,9773843
rec-206-dup-0,NaN,lombardi,36,yerralee road,leisure living vlge,carlsruhe,3149,qld,19870919,1613132


In [58]:
from recordlinkage.index import Full
dupe_indexer = recordlinkage.Index()
#full for all possible combinations
#dupe_indexer.full()

#blocking
dupe_indexer.block('given_name')

#sorted 
#dupe_indexer.sortedneighbourhood(left_on='given_name')

dupe_candidate_links = dupe_indexer.index(data)

In [61]:
print(len(dupe_candidate_links))

2082


In [73]:
compare_dupes = recordlinkage.Compare()
compare_dupes.exact('given_name','given_name',label='given_name')
compare_dupes.string('surname','surname', method='jarowinkler', threshold=0.85,label='surname')
compare_dupes.exact('date_of_birth','date_of_birth',label='date_of_birth')
compare_dupes.exact('suburb','suburb',label='suburb')
compare_dupes.exact('state','state',label='state')
compare_dupes.string('address_1','address_1',threshold=0.85,label='address_1')
dupe_features = compare_dupes.compute(dupe_candidate_links,data)

In [74]:
dupe_features.head(10)

given_name  surname  date_of_birth  suburb  \
rec_id_1      rec_id_2                                                    
rec-183-dup-0 rec-122-org             1      0.0              0       0   
rec-248-org   rec-122-org             1      0.0              0       0   
              rec-183-dup-0           1      0.0              0       0   
rec-122-dup-0 rec-122-org             1      1.0              1       1   
              rec-183-dup-0           1      0.0              0       0   
              rec-248-org             1      0.0              0       0   
rec-469-org   rec-122-org             1      0.0              0       0   
              rec-183-dup-0           1      0.0              0       0   
              rec-248-org             1      0.0              0       0   
              rec-122-dup-0           1      0.0              0       0   

                             state  address_1  
rec_id_1      rec_id_2                         
rec-183-dup-0 rec-122-org        0        0.0  
rec-248-org   rec-122-org        1        0.0  
              rec-183-dup-0      0        0.0  
rec-122-dup-0 rec-122-org        1        1.0  
              rec-183-dup-0      0        0.0  
              rec-248-org        1        0.0  
rec-469-org   rec-122-org        0        0.0  
              rec-183-dup-0      1        0.0  
              rec-248-org        0        0.0  
              rec-122-dup-0      0        0.0

In [75]:
dupe_features.describe()

,given_name,surname,date_of_birth,suburb,state,address_1
count,2082.0,2082.000000,2082.000000,2082.000000,2082.000000,2082.000000
mean,1.0,0.144092,0.139289,0.108549,0.327089,0.133045
std,0.0,0.351268,0.346331,0.311148,0.469263,0.339705
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.0,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [76]:
#sum the comparison results
dupe_features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0     142
5.0     145
4.0      30
3.0       9
2.0     376
1.0    1380
dtype: int64

In [77]:
matches = dupe_features[dupe_features.sum(axis=1)>3]

print (len(matches))
matches.head(10)

317


,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-122-dup-0,rec-122-org,1,1.0,1,1,1,1.0
rec-183-org,rec-183-dup-0,1,1.0,1,1,1,1.0
rec-248-dup-0,rec-248-org,1,1.0,1,1,1,1.0
rec-373-dup-0,rec-373-org,1,1.0,1,1,1,1.0
rec-10-org,rec-10-dup-0,1,1.0,1,1,1,1.0
rec-342-dup-0,rec-342-org,1,1.0,0,1,1,1.0
rec-397-org,rec-397-dup-0,1,1.0,1,1,1,0.0
rec-472-org,rec-472-dup-0,1,1.0,1,1,1,0.0
rec-330-org,rec-330-dup-0,1,0.0,1,1,1,0.0
